In [31]:
import requests
import pandas as pd
import sqlite3
import logging

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para criar a tabela no banco de dados SQLite (se ainda não existir)
def criar_tabela():
    with sqlite3.connect('dados_pontos.db') as conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS onibus (
                NOME TEXT,
                NUM TEXT,
                Latitude REAL,
                Longitude REAL,
                SEQ INTEGER,
                GRUPO TEXT,
                SENTIDO TEXT,
                TIPO TEXT,
                ID_do_Itinerario INTEGER,
                COD TEXT
            )
        ''')
        conn.commit()

# Função para buscar e processar dados da linha de ônibus
def buscar_e_processar_dados(linha):
    url = f'https://transporteservico.urbs.curitiba.pr.gov.br/getPontosLinha.php?linha={linha:03}&c=821f0'
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        dados_json = response.json()

        # Verificar se os dados estão vazios
        if not dados_json:
            return None

        # Criar DataFrame a partir dos dados
        df = pd.DataFrame.from_records(dados_json)

        # Substituir vírgulas por pontos e converter tipos
        df['Latitude'] = df['LAT'].str.replace(',', '.').astype(float)
        df['Longitude'] = df['LON'].str.replace(',', '.').astype(float)
        df['SEQ'] = df['SEQ'].astype(int)
        df['ID_do_Itinerario'] = df['ITINERARY_ID'].astype(int)
        
        # Adicionar coluna COD
        df['COD'] = linha

        return df[['NOME', 'NUM', 'Latitude', 'Longitude', 'SEQ', 'GRUPO', 'SENTIDO', 'TIPO', 'ID_do_Itinerario', 'COD']]

    except requests.RequestException as e:
        logging.error(f"Erro na requisição para a linha {linha}: {e}")
    except Exception as e:
        logging.error(f"Erro ao processar a linha {linha}: {e}")
    return None

# Função para carregar a tabela de ônibus em um DataFrame
def carregar_tabela_para_dataframe(nome_tabela, db='dados_pontos.db'):
    try:
        with sqlite3.connect(db) as conn:
            cursor = conn.cursor()

            # Verificar se a tabela existe
            cursor.execute("""
                SELECT name FROM sqlite_master WHERE type='table' AND name=?;
            """, (nome_tabela,))
            resultado = cursor.fetchone()

            if resultado:
                # Carregar a tabela em um DataFrame
                df = pd.read_sql(f'SELECT * FROM {nome_tabela}', conn)
            else:
                df = pd.DataFrame()  # Retornar DataFrame vazio se a tabela não existir

        return df
    except Exception as e:
        logging.error(f"Erro ao carregar a tabela {nome_tabela}: {e}")
        return pd.DataFrame()

# Função para gravar os dados no banco de dados SQLite
def gravar_no_banco(df_result):
    try:
        with sqlite3.connect('dados_pontos.db') as conn:
            df_result.to_sql('onibus', conn, if_exists='append', index=False)
            logging.info(f"{len(df_result)} novos pontos gravados no banco de dados.")
    except Exception as e:
        logging.error(f"Erro ao gravar no banco de dados: {e}")

# Função para processar as linhas de ônibus de forma sequencial (sem threads)
def processar_linhas_sequencialmente(linhas_unicas, df_onibus):
    result_list = []

    for linha in linhas_unicas:
        df_result = buscar_e_processar_dados(linha)
        if df_result is not None:
            result_list.append(df_result)

    # Verificar e concatenar resultados
    if result_list:
        df_concatenado = pd.concat(result_list, ignore_index=True)
        logging.info(f"Total de linhas processadas: {len(df_concatenado)}")

        # Inicializar df_novos_pontos
        df_novos_pontos = pd.DataFrame()

        # Verificar se existem novos pontos (baseado em COD)
        if not df_onibus.empty:
            df_novos_pontos = df_concatenado[~df_concatenado['COD'].isin(df_onibus['COD'])]
        else:
            df_novos_pontos = df_concatenado.copy()

        # Gravar novos pontos no banco de dados, se houver
        if not df_novos_pontos.empty:
            gravar_no_banco(df_novos_pontos)
        else:
            logging.info("Nenhum novo ponto para gravar.")

# Inicializar o processamento sequencial
def iniciar_processamento():
    linhas_unicas = range(0, 1000)  # Exemplo de intervalo de linhas
    criar_tabela()  # Garantir que a tabela está criada

    try:
        # Carregar tabela do banco de dados em um DataFrame
        df_onibus = carregar_tabela_para_dataframe('onibus')

        # Processar as linhas de forma sequencial (sem threads)
        processar_linhas_sequencialmente(linhas_unicas, df_onibus)
    except Exception as e:
        logging.error(f"Erro durante o processamento: {e}")
    finally:
        logging.info("Processamento finalizado.")

# Chamar a função principal para iniciar o processamento
if __name__ == "__main__":
    iniciar_processamento()


2024-10-16 23:12:53,202 - ERROR - Erro na requisição para a linha 103: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-16 23:13:36,746 - ERROR - Erro na requisição para a linha 204: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-16 23:15:35,703 - ERROR - Erro na requisição para a linha 540: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-16 23:15:53,441 - ERROR - Erro na requisição para a linha 564: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-16 23:16:30,756 - ERROR - Erro na requisição para a linha 570: HTTPSConnectionPool(host='transporteservico.urbs.curitiba.pr.gov.br', port=443): Read timed out. (read timeout=10)
2024-10-16 23:16:49,693 - ERROR - Erro na requisição para a linha 572:

In [24]:
df_concatenado

,NOME,NUM,Latitude,Longitude,SEQ,GRUPO,SENTIDO,TIPO,ID do Itinerario,COD
0,"Rua Antônio Schiebel, 1624 - Boqueirão",150335,-25.500940,-49.242273,10,,Terminal Carmo,Chapéu chinês,737,521
1,"Rua Júlio Zandoná, 755 - Alto Boqueirão",150342,-25.522215,-49.249518,1,,Terminal Carmo,Chapéu chinês,737,521
2,"Rua Cascavel, 328 - Boqueirão",150749,-25.503470,-49.251600,7,,Nivaldo Braga,Chapéu chinês,13833,521
3,"Rua Antônio Schiebel, 1957 - Boqueirão",150784,-25.502059,-49.245364,3,,Nivaldo Braga,Chapéu chinês,13833,521
4,"Rua Max Schubert, 195 - Alto Boqueirão",150340,-25.520898,-49.253236,16,,Nivaldo Braga,Chapéu chinês,13833,521
5,"Rua Jorge Brey, 343 - Xaxim",150897,-25.522123,-49.253694,17,,Nivaldo Braga,Placa em poste,13833,521
6,"Rua Paulo Setúbal, 4103 - Hauer",150044,-25.512567,-49.244580,5,,Terminal Carmo,Chapéu chinês,737,521
7,"Rua Paulo Setúbal, 3896 - Hauer",150045,-25.511051,-49.245313,6,,Terminal Carmo,Domus,737,521
8,"Rua Paulo Setúbal, 3377 -Hauer",150046,-25.506771,-49.247379,7,,Terminal Carmo,Chapéu chinês,737,521
9,"Rua Max Schubert, 495 - Alto Boqueirão",150339,-25.521198,-49.250695,15,,Nivaldo Braga,Chapéu chinês,13833,521
